## Hands on Building of a Geographic Model
### Facility Location Problems - Group Exercise

Working in your PSGs, use what you have just observed (and learned) to tackle the following case study. In the cells below there will be omissions (denoted with <span style="color:red">**\*\***</span>) that need to be completed correctly to move on to the next step.


#### Case Study: TIA Clinics in Cornwall

Transient Ischaemic Attacks (TIAs) are a minor type of stroke (disruption of blood flow to the brain) where symptoms may last for up to 24 hours. TIAs can be important predictors of major strokes. These can can be fatal or cause substantial disability. Therefore, it is important for patients to have suitable and equitable access to a specialist TIA clinic shortly after their symptoms occur.

This case study is set in Cornwall where TIA patients are currently assessed and treated at five clinic locations. Due to funding constraints not all locations are open every day. The locations are:

* Royal Cornwall Hospital Trust (RCHT)
* Cambourne Redruth Community Hospital (CRCH)
* St. Austell Community Hospital (SACH)
* Bodmin Community Hospital (BODMIN)
* West Cornwall Hospital (WCH)

Local NHS commissioners are considering the future of the service. They feel that five clinic locations is leading to variation in the quality of patient treatment and would like to **consolidate to three locations.** 

*They have asked for our help in making and supporting their decision making.*

The data provided comprises of:

* *tia_travel_matrix.csv* - Travel time matrix from different locations
* *tia_attendances.csv* - Number of TIA clinic attendances per LSOA

### Library Imports

Do you remember which *standard* libraries were imported during the demonstration? Import them again to complete the rest of this exercise. 

In [ ]:
# update the code below
import **
import **
import **


# Additionally, we'll also use....
# combinations from the itertools library allows us to enumerate all
# solutions (for small instances).
from itertools import combinations

In addition, run the cell below to load in the FacilityLocationObjective class.

In [ ]:
# Tweaked WeightedAverageObjective from Metapy package
# https://github.com/health-data-science-OR/healthcare-logistics/tree/master/optimisation/metapy
# Credit: Tom Monks

class FacilityLocationObjective:
    '''
    Encapsulates logic for calculation of
    metrics in a simple facility location problem

    Demand and travel matrices must have a common column

    demand: pd.dataframe:  Two column dataframe. One column should be labels for the
    demand locations (e.g. LSOA identifiers, postcodes). Second column should contain
    demand figures of some kind (e.g. number of historical cases)
    If demand assumed to be equal, all values in this column could be 1.

    travel_matrix: pd.dataframe: dataframe with columns representing sites
    and rows representing locations demand will come from.
    One column should be labels for the demand locations (e.g. LSOA identifiers, postcodes).
    All other values will be either distance or time in float form.
    No additional columns of information must be included or they will be used as part of the
    calculation of the lowest-cost solution, which may lead to incorrect results.
    '''
    def __init__(self, demand, travel_matrix, merge_col, demand_col):
        '''
        Store the demand and travel times

        Args:
            demand: pd.DataFrame:

            travel_matrix: pd.DataFrame:
        '''
        self.demand = demand.set_index(merge_col)
        self.travel_matrix = travel_matrix.set_index(merge_col)
        self.demand_col = demand_col


    def evaluate_solution(self, site_list):
        '''
        Calculates the

        Args:
            site_list: list: column indices of solution to evaluate
                            (to apply to travel matrix)

            merge_col: str: name of shared column in demand df and travel time
                    df that contains labels for areas that can be merged on (e.g. "lsoa")

            n_patients_or_referrals_col: str: name of column containing numeric data
                                            (e.g. "n_patients", "referrals")

        Returns:
            Pandas dataframe to pass to evaluation functions

        '''

        active_facilities = self.travel_matrix.iloc[:, site_list].copy()

        # Assume travel to closest facility
        # Need to drop the column that contains
        active_facilities['min_cost'] = active_facilities.min(axis=1)


        # Merge demand and travel times into a single DataFrame
        problem = self.demand.merge(active_facilities,
                                    left_index=True, right_index=True,
                                    how='inner')

        return problem.reset_index()


    def generate_solution_metrics(self, site_list):
        '''
        Calculates the weighted average travel time for selected sites

        Args:
            site_list: list or np.array: A list of site IDs as a list or array (e.g. [0, 3, 4])
            merge_col: string: The column name to use for merging the data.
            n_patients_or_referrals_col: string: The column name to use for the number of patients or referrals.

        Returns:
            A tuple containing the problem and the maximum travel time.
        '''
        problem = self.evaluate_solution(site_list)

        # Return weighted average
        weighted_average = np.average(problem['min_cost'], weights=problem[self.demand_col])
        unweighted_average = np.average(problem['min_cost'])
        max_travel = np.max(problem['min_cost'])

        return {
            'site_indices': site_list,
            'site_names': ", ".join(self.travel_matrix.columns[site_list].tolist()),
            'weighted_average': weighted_average,
            'unweighted_average': unweighted_average,
            'max': max_travel,
            'problem_df': problem
        }

### Data Imports
<a name="data_imports">
   Code to Import Data
</a>

Now we need to import the travel matrix file named `tia_travel_matrix.csv` and check that the file has been read correctly. Define the index as `sector`

In [ ]:
# your code goes here ...
travel_matrix = pd.read_csv('../data/**.csv',
                            index_col='sector')
travel_matrix.head()

Next, import the number of `tia_attendances.csv` file (in the same location). Again, check it's been read correctly and remember to set the index to `sector`.

In [ ]:
# your code goes here ...
attends = pd.read_csv('../data/**.csv',
                            index_col='sector')
attends.head()

Now we will create our location problem object.

In [ ]:
facility_location_problem = FacilityLocationObjective(
    demand= , ## YOUR CODE HERE
    travel_matrix= , ## YOUR CODE HERE
    merge_col="sector",
    demand_col="n_patients"
    )

### Constructing and evaluating a Random Solution

From the above you'll have seen that you've been provided with a travel matrix containing 5 clinic locations. For this case study, the 'solution' will comprise of 3 proposed clinics.

Now it's time to generate a random solution. First, run the code in the cell below to define the random_solution function.

In [ ]:
def random_solution(n_candidates, p, random_seed=None):
    '''
    Helper function to generate a random solution

    Params
    ------
    n_candidates : int
        The number of candidate locations where you could place
        clinics (facilities).

    p : int
        The number of clinics to place.

    random_seed : int (Default=None)
        Random seed for reproducibility.

    Returns
    -------

    Vector (np.array) of length p
    '''
    # create a random number generator
    rng = np.random.default_rng(seed=random_seed)

    # sample without replacement
    solution = []
    while len(solution) < p:
        candidate = rng.integers(0, n_candidates)
        if candidate not in solution:
            solution.append(candidate)

    return np.array(solution)

Next, fill in the blanks in the use of the function below to generate a single solution

In [ ]:
random_solution_1 = random_solution(
    n_candidates= , ## YOUR CODE HERE
    p= , ## YOUR CODE HERE
    random_seed=42
)

Now let's evaluate this solution.

In [ ]:
facility_location_problem.evaluate_solution(
    site_list = ## YOUR CODE HERE
)

This time, let's return the metrics from this solution.

In [ ]:
metrics_random_solution_1 = facility_location_problem.generate_solution_metrics(
    site_list = ## YOUR CODE HERE
)

metrics_random_solution_1

Convert this into a dataframe. Is this easier to read?

In [ ]:
## YOUR CODE HERE

### Create all Possible Combinations

The `all_combinations` function has been provided below.

**Question** Who can remember what a list comprehension is? Where is it used in the function, below?

In [ ]:
def all_combinations(n_facilities, p):
    '''
    n_facilities : int
        The number of candidate locations where you could place facilities (clinics).

    p : int
        The number of clinics to place.

    Returns
    -------

    Returns all p sized combinations of an array containing
    indicies 0 to n_facilties - 1
    '''
    facility = np.arange(n_facilities, dtype=np.uint8)
    return [np.array(a) for a in combinations(facility, p)]

Use the function above to print the total number of possible combinations (remember - we're looking for 3 locations from the existing 5 clinics).

In [ ]:
# your code goes here ...
combinations = ## YOUR CODE HERE
print(combinations)


Print the total number of combinations.

In [ ]:
number_of_combinations = ## YOUR CODE HERE

print(f"The total number of combinations is {number_of_combinations}")

Take a look at one of the combinations. 

What combination of clinics is propsed in the 5th element of the array of combinations?

In [ ]:
## YOUR CODE HERE
**[4]

### Brute-forcing the solution

Assess all of the possible combinations.

Make sure you add the output of each round of the loop to the list `results`. This has been set up as an empty list. 


In [ ]:
# your code goes here ...
results = []

for possible_solution in combinations:
    **

Convert `results` into a single dataframe.

In [ ]:
results_df = **

Run the code below to print the optimal solution.

In [ ]:
# your code goes here ...
optimal_solution_wa = pd.DataFrame(results_df).sort_values('weighted_average').head(1)['site_names'].values


print('The optimal combination of sites'+\
      f' is {optimal_index}')

### Graphical Representation of the Results

Generate a bar chart of all possible solution costs.

You can choose to use any library you like - the lecture used plotly express, but you could use plotnine, matplotlib, or seaborn if you prefer. 

Your chart should include the following elements:
* All possible clinic combinations represented as bars
* Lowest cost solution should be coloured in <span style="color: green;">**green**</span> 
* Title
* x-axis label
* y-axis label


In [ ]:
# your code goes here ...

### Visual Display of Travel Times on the Map

Use the files `LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_-6793269404754981576.geojson` and `tia_sites.csv` to generate a travel time map for the best solution.

In [ ]:
## YOUR CODE HERE

Now try using the subplots feature of matplotlib to create a plot for every possible combination of clinics. 

Consider the order you might want the plots in.

What additional information is it useful to include in the subplot titles?

In [ ]:
## YOUR CODE HERE

## Bonus Activities

### 1. Calculate the best solution across the weighted average travel times, unweighted average travel times, and the max travel time. 

How might you rank the outputs?

How might you present the results?

In [ ]:
## YOUR CODE HERE

### 2. How might you account for LSOAs without any demand?

You may have noticed that some LSOAs appear to have no demand in the time period shown. 

Think about how this might affect the results. 

How might you create an additional demand dataframe that ensures every LSOA in Cornwall is included? 

In [ ]:
## YOUR CODE HERE

### 3. What's the impact of hospitals across the border?

Let's imagine the Derriford hospital in Plymouth - just across the border - can also provide TIA services.

Use your routingpy skills from the previous to grab an additional column of travel data.

(Note that the existing travel time data is quite old - so they may not be perfectly comparable, but are fine for the purposes of this exercise!)

Add this additional data to your existing travel time dataset. 

Redo the analysis above and compare the outputs. Does adding in Derriford significantly change which clinics we would recommend to keep?

In [ ]:
## YOUR CODE HERE

To take this even further, switch to keeping 4 clinic open - but one of them will *always* be Derriford. 

How could you create these combinations? 

Rerun your analysis.

In [ ]:
## YOUR CODE HERE